In [ ]:
import pandas as pd
#import gseapy as gp
import matplotlib.pyplot as plt

from goatools import obo_parser
import wget
import os

import seaborn as sns
import mygene
import numpy as np
import collections
import pycrosstalker

from goatools.obo_parser import GODag
from goatools.associations import read_gaf

In [ ]:
which_os = "windows11"

In [ ]:
#RCC = pd.read_csv("/home/larissa/Documents/Masterarbeit/RCC_results/liana_log_norm/no_subset_glom_removed/T_lr_ready.csv", index_col=0)
#MF = pd.read_csv("/home/larissa/Documents/Masterarbeit/MF_results/liana/all_celltypes/MF_lr_ready.csv", index_col=0)

#RCC = pd.read_csv("D:\\studium\\Masterarbeit\\RCC_results\\liana_log_norm\\no_subset_glom_removed\\T_lr_ready.csv", index_col=0)
#MF = pd.read_csv("D:\\studium\\Masterarbeit\\MF_results\\liana\\all_celltypes\\MF_lr_ready.csv", index_col=0)


RCC = pd.read_csv("C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\RCC_results\\liana_log_norm\\no_subset_glom_removed\\T_lr_ready.csv", index_col = 0)
MF = pd.read_csv("C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\MF_results\\liana\\all_celltypes\\MF_lr_ready.csv", index_col = 0)


RCC["lr_pair"] = RCC["gene_A"] + "_" + RCC["gene_B"]
MF["lr_pair"] = MF["gene_A"] + "_" + MF["gene_B"]

RCC["cluster_pair"] = RCC["source"] + "@" + RCC["target"]
MF["cluster_pair"] = MF["source"] + "@" + MF["target"]



In [ ]:
#RBConfig res 1.25 after removing glom endo
cluster_dict = {"Monocytes" : ["MF_CD16_monocytes", "MF_Classical_monocytes", "MF_EBM_prog", "MF_GMPs",
    "MF_Inflammatory_CD14_monocytes", "MF_Late_Neutro_Prog", "MF_Macrophages",
    "MF_Neutrophils", "MF_SPP1_Macrophages", "MF_cDC1", "MF_cDC2", "RCC_Classical monocytes", "RCC_Non-classical monocytes", "RCC_TAM 1", "RCC_TAM 2", "RCC_TAM 3", "RCC_TAM 4"],
"Lymphocytes" : ["MF_B_cells", "MF_CD8_T_cells","MF_NK_T_cells", "RCC_B cells",
    "RCC_CD8 T cells", "RCC_Cytotoxic T cells", "RCC_IGHG-high plasma cells",
    "RCC_NK cells", "RCC_Plasma cells", "RCC_Regulatory T cells",
    "RCC_Resting/memory T cells"],#, "RCC_Proximal tubule"],
"Tumor_prog" : ["MF_B_cell_prog", "MF_Plasma_cells",
                 "MF_Early_Neutro_Prog", "MF_HSPCs", "MF_MEPs", "MF_MK_prog",
    "RCC_DCT/CNT", "RCC_Epithelial progenitor-like cells", "RCC_Principal cells",
    "RCC_Tumor cells 1", "RCC_Tumor cells 2", "RCC_Tumor cells 3", "RCC_tAL of LOH"],
"Endothelial" : [ "MF_pDCs", "MF_Arterial_EC", "MF_Erythroid_cells", "MF_Late_Erythroid_Prog",
    "MF_Sinusoidal_EC", "RCC_AVR", "RCC_DVR", "RCC_Tumor AVR-like vasculature",
    "RCC_Tumor vasculature 1", "RCC_Tumor vasculature 2", "RCC_Tumor vasculature 3", "RCC_Tumor vasculature 4"],
"Stromal" : ["MF_Adipo_CAR", "MF_Mature_MKs", "MF_OLCs", "MF_Osteoblasts", "RCC_Mesangial/vSMCs",
    "RCC_Myofibroblasts", "RCC_vSMCs"],
"Mast_cells" : ["MF_Mast_cells", "RCC_Mast cells"]}

In [ ]:
def build_dicts(df, clusters, opt = "all"):

        if opt == "all":
            sub_df = df[df["source"].isin(clusters) | df["target"].isin(clusters)]
        elif opt == "intra":
            sub_df = df[df["source"].isin(clusters) & df["target"].isin(clusters)]
            #sub_df = sub_df[sub_df["source"] != sub_df["target"]]
        elif opt == "inter":
            sub_df = df[df["source"].isin(clusters) | df["target"].isin(clusters)]
            sub_df = sub_df[~((sub_df["source"].isin(clusters)) & (sub_df["target"].isin(clusters)))]
        #print(sub_df)

        source_dict = {cl: [] for cl in clusters}
        target_dict = {cl: [] for cl in clusters}
        
        for _, row in sub_df.iterrows():
            if row["source"] in source_dict:
                source_dict[row["source"]].append(row["lr_pair"])
            if row["target"] in target_dict:
                target_dict[row["target"]].append(row["lr_pair"])
        return source_dict, target_dict

def threshold_intersection(sets_list, threshold):
       
                n_sets = len(sets_list)
                min_count = max(1, int(threshold * n_sets))  # at least 1
                counter = collections.Counter()
                for s in sets_list:
                    counter.update(s)
                return {elem for elem, count in counter.items() if count >= min_count}

    
def build_dicts_associated_gene(df, clusters, opt = "all"):

        if opt == "all":
            sub_df = df[df["source"].isin(clusters) | df["target"].isin(clusters)]
        elif opt == "intra":
            sub_df = df[df["source"].isin(clusters) & df["target"].isin(clusters)]
        elif opt == "inter":
            sub_df = df[df["source"].isin(clusters) | df["target"].isin(clusters)]
            sub_df = sub_df[~sub_df["source"].isin(clusters) & sub_df["target"].isin(clusters)]
        source_dict = {cl: [] for cl in clusters}
        target_dict = {cl: [] for cl in clusters}
        
        for _, row in sub_df.iterrows():
            if row["source"] in source_dict:
                source_dict[row["source"]].append(row["gene_A"])
            if row["target"] in target_dict:
                target_dict[row["target"]].append(row["gene_B"])
        return source_dict, target_dict

In [ ]:
#filter LR sets
def get_intra_inter(opt, opt2, opt3):
    all_interactions_list = []
    all_interactions_source_list = []
    all_interactions_target_list = []
    unique_per_cluster = []
    unique_per_cluster_source = []
    unique_per_cluster_target = []
    test_clust = []
    all_interactions_dict = {}

    RCC_source_dict = {}
    RCC_target_dict = {}
    MF_source_dict = {}
    MF_target_dict = {}
    
    for i in cluster_dict: 
        tst_clust = []
        all_interactions = []
        all_interactions_source = []
        all_interactions_target = []
        
        for clust in cluster_dict[i]:
            tst_clust.append(clust)
            
        print(tst_clust)
        tst_clust = list(map(lambda x:  x.removeprefix("RCC_").removeprefix("MF_"), tst_clust))
        
        if opt3 == "intra":
            if opt2 == "a" or opt2 =="a2" or opt2 == "b" or opt2 =="e":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust, opt = "intra")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust, opt = "intra")

            if opt2 == "c":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust, opt = "intra")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust, opt = "intra")
                
                downstreamL_RCC, _ = build_dicts_associated_gene(RCC, tst_clust, opt = "intra")
                downstreamL_MF, _ = build_dicts_associated_gene(MF, tst_clust, opt = "intra")

            if opt2 == "d":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts_associated_gene(RCC, tst_clust, opt = "intra")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts_associated_gene(MF, tst_clust, opt = "intra")

        elif opt3 == "inter":
            if opt2 == "a"  or opt2 =="a2" or opt2 == "b" or opt2 =="e":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust, opt = "inter")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust, opt = "inter")

            if opt2 == "c":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust, opt = "inter")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust, opt = "inter")

                downstreamL_RCC, _ = build_dicts_associated_gene(RCC, tst_clust, opt = "all")
                downstreamL_MF, _ = build_dicts_associated_gene(MF, tst_clust, opt = "all")

            if opt2 == "d":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts_associated_gene(RCC, tst_clust, opt = "inter")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts_associated_gene(MF, tst_clust, opt = "inter")
        #print("clust dict source RCC", clust_dict_source_RCC)
        RCC_source_sets =  {cl: set(clust_dict_source_RCC[cl]) for cl in tst_clust if len(clust_dict_target_RCC[cl]) != 0}
        MF_source_sets = {cl: set(clust_dict_source_MF[cl]) for cl in tst_clust if len(clust_dict_target_MF[cl]) != 0}
        #print("rcc source sets", RCC_source_sets)
        
        RCC_target_sets =  {cl: set(clust_dict_target_RCC[cl]) for cl in tst_clust if len(clust_dict_target_RCC[cl]) != 0}
        MF_target_sets = {cl: set(clust_dict_target_MF[cl]) for cl in tst_clust if len(clust_dict_target_MF[cl]) != 0}

        if opt == "set" or opt == "unique":

            if opt2 == "c":

                downstreamL_RCC = {cl: set(downstreamL_RCC[cl]) for cl in tst_clust if len(downstreamL_RCC[cl]) != 0}
                downstreamL_MF = {cl: set(downstreamL_MF[cl]) for cl in tst_clust if len(downstreamL_MF[cl]) != 0}
                #print(downstreamL_RCC)

                RCC_target_sets= {
                cl: RCC_target_sets.get(cl, set()) | downstreamL_RCC.get(cl, set())
                    for cl in downstreamL_RCC.keys() if len(downstreamL_RCC[cl]) != 0}
                #print(downstreamL_RCC)

                MF_target_sets = {
                cl: MF_target_sets.get(cl, set()) | downstreamL_MF.get(cl, set())
                for cl in downstreamL_MF.keys()  if len(downstreamL_MF[cl]) != 0}
                
                #RCC_target_sets = {k: set(v) for k, v in RCC_target_sets.items()}
                #MF_target_sets = {k: set(v) for k, v in MF_target_sets.items()}
                

            RCC_source_sets_interactions = []
            RCC_target_sets_interactions = []
            RCC_total_interactions = []
            if len(RCC_target_sets) >= 1: 
                RCC_source_sets_interactions =  set.intersection(*RCC_source_sets.values())
                RCC_target_sets_interactions =  set.intersection(*RCC_target_sets.values())
                RCC_total_interactions = RCC_source_sets_interactions.union(RCC_target_sets_interactions)
                #print(RCC_target_sets)
                #print("RCC_source_sets_interactions", RCC_source_sets_interactions)
                #print("RCC_target_sets_interactions", RCC_target_sets_interactions)
                

            MF_source_sets_interactions = []
            MF_target_sets_interactions = []
            MF_total_interactions = []
            if len(MF_target_sets) >= 1: 
                MF_source_sets_interactions =  set.intersection(*MF_source_sets.values())
                MF_target_sets_interactions =  set.intersection(*MF_target_sets.values())
                MF_total_interactions = MF_source_sets_interactions.union(MF_target_sets_interactions)
                #print("MF_source_sets_interactions", MF_source_sets_interactions)
                #print("MF_target_sets_interactions", MF_target_sets_interactions)
            if opt2 == "a" or opt2 =="d":
                if len(RCC_total_interactions) > 1 and len(MF_total_interactions) > 1: 
                    all_interactions = RCC_total_interactions.intersection(MF_total_interactions)
                else:
                    all_interactions = set()
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

            if opt2 == "b"  or opt2 == "d2":
                if len(MF_source_sets_interactions) > 1 or len(RCC_source_sets_interactions) > 1: 
                    all_interactions_source = RCC_source_sets_interactions.intersection(MF_source_sets_interactions)
                    all_interactions_target = RCC_target_sets_interactions.intersection(MF_target_sets_interactions)
                else:
                    all_interactions = set()

                all_interactions_source_list.append(all_interactions_source)
                all_interactions_target_list.append(all_interactions_target)
                test_clust.append(tst_clust)
                
            if opt2 == "c":
                if len(MF_target_sets_interactions) > 1 and len(RCC_target_sets_interactions) > 1: 
                    all_interactions = RCC_target_sets_interactions.intersection(MF_target_sets_interactions)  
                else:
                    all_interactions = set()
                
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

            if opt2 == "e":
                if len(MF_target_sets_interactions) > 1 and len(RCC_target_sets_interactions) > 1: 
                    all_interactions = RCC_target_sets_interactions.intersection(MF_target_sets_interactions)  
                else:
                    all_interactions = set()
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)
            
            #print(RCC_total_interactions, MF_total_interactions)
            #print(RCC_target_sets_interactions, MF_target_sets_interactions)

            if opt2 =="a2":
                
                
                RCC_source_dict[i] = RCC_source_sets_interactions
                RCC_target_dict[i] = RCC_target_sets_interactions
                MF_source_dict[i] = MF_source_sets_interactions
                MF_target_dict[i] = MF_target_sets_interactions
                test_clust.append(tst_clust)
       

        ###########################################################################################################  
        if opt == "threshold":

            if opt2 == "c":

                downstreamL_RCC = {cl: set(downstreamL_RCC[cl]) for cl in tst_clust if len(downstreamL_RCC[cl]) != 0}
                downstreamL_MF = {cl: set(downstreamL_MF[cl]) for cl in tst_clust if len(downstreamL_MF[cl]) != 0}
                #print(downstreamL_RCC)

                RCC_target_sets= {
                cl: RCC_target_sets.get(cl, set()) | downstreamL_RCC.get(cl, set())
                    for cl in downstreamL_RCC.keys() if len(downstreamL_RCC[cl]) != 0}
                #print(downstreamL_RCC)
                #print("rcc target sets", RCC_target_sets)
                MF_target_sets = {
                cl: MF_target_sets.get(cl, set()) | downstreamL_MF.get(cl, set())
                for cl in downstreamL_MF.keys()  if len(downstreamL_MF[cl]) != 0}
                #print("MF target sets", MF_target_sets)
                #RCC_target_sets = {k: set(v) for k, v in RCC_target_sets.items()}
                #MF_target_sets = {k: set(v) for k, v in MF_target_sets.items()}
                
            RCC_source_sets_list = list(RCC_source_sets.values())
            MF_source_sets_list = list(MF_source_sets.values())
            RCC_target_sets_list = list(RCC_target_sets.values())
            MF_target_sets_list = list(MF_target_sets.values())

            threshold = 0.7  # %

            source_interactions_RCC = threshold_intersection(RCC_source_sets_list, threshold) if len(RCC_source_sets_list) > 0 else set()
            target_interactions_RCC = threshold_intersection(RCC_target_sets_list, threshold) if len(RCC_target_sets_list) > 0 else set()

            source_interactions_MF = threshold_intersection(MF_source_sets_list, threshold) if len(MF_source_sets_list) > 0 else set()
            target_interactions_MF = threshold_intersection(MF_target_sets_list, threshold) if len(MF_target_sets_list) > 0 else set()

            all_interactions_RCC = source_interactions_RCC.union(target_interactions_RCC) if len(source_interactions_RCC) > 0 else set()
            all_interactions_MF = source_interactions_MF.union(target_interactions_MF) if len(source_interactions_MF) > 0 else set()
            #print("target interactions RCC",target_interactions_RCC)
            #print("target interactions MF",target_interactions_MF)

            if opt2 == "a" or opt2 == "d":
                all_interactions = all_interactions_RCC.intersection(all_interactions_MF) if (len(all_interactions_MF) > 0 & len(all_interactions_RCC)) else set()
                #all_interactions = list(threshold_intersection([all_interactions_RCC, all_interactions_MF]) if len(all_interactions_MF) > 0 else set())
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)
        
            if opt2 == "b" or opt2 == "d2":
                if len(source_interactions_MF) > 1 or len(source_interactions_RCC) > 1: 
                    all_interactions_source = source_interactions_RCC.intersection(source_interactions_MF)
                    all_interactions_target = target_interactions_RCC.intersection(target_interactions_MF)
                else:
                    all_interactions = set()

                all_interactions_source_list.append(all_interactions_source)
                all_interactions_target_list.append(all_interactions_target)
                test_clust.append(tst_clust)

            if opt2 =="c":
                if len(target_interactions_MF) > 1 and len(target_interactions_RCC) > 1: 
                    all_interactions = target_interactions_RCC.intersection(target_interactions_MF)  
                else:
                    all_interactions = set()
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

     ######################
    if opt == "unique":

            if opt2 == "a" or opt2 == "d" or opt2 == "c":
                for i, clustergenes in enumerate(all_interactions_list):
                    #print(len(cluster))
                    if len(clustergenes) > 0:
                        #print("clustergenes", clustergenes)
                        set_i = set(clustergenes)  # interactions in this cluster
                        #print("set_i", set_i)
                        others = set().union(*[all_interactions_list[j] for j in range(len(all_interactions_list)) if j != i])
                        unique = set_i - others  # LRs only in this cluster
                        #print("unique", unique)
                        unique_per_cluster.append(unique)
                    else:
                        unique_per_cluster.append(set())

            if opt2 == "b" or opt2 == "d2":
                 
                for i, cluster in enumerate(all_interactions_source_list):
                    set_i = set(cluster)  # interactions in this cluster
                    others = set().union(*[all_interactions_source_list[j] for j in range(len(all_interactions_source_list)) if j != i])
                    unique = set_i - others  # LRs only in this cluster
                    unique_per_cluster_source.append(unique)

                for i, cluster in enumerate(all_interactions_target_list):
                    set_i = set(cluster)  # interactions in this cluster
                    others = set().union(*[all_interactions_target_list[j] for j in range(len(all_interactions_target_list)) if j != i])
                    unique = set_i - others  # LRs only in this cluster
                    unique_per_cluster_target.append(unique)

            if opt2 == "a2":
                
                source_dict_unique = {}
                target_dict_unique = {}
                
                

                if len(RCC_source_dict) > 1 and len(MF_source_dict) > 1:    
                    source_dict = {
                        cl: RCC_source_dict[cl].intersection(MF_source_dict.get(cl, set()))
                        for cl in RCC_source_dict.keys()
                    }

            
                if len(RCC_target_dict) > 1 and len(MF_target_dict) > 1:    
                    target_dict = {
                        cl: RCC_target_dict[cl].intersection(MF_target_dict.get(cl, set()))
                        for cl in RCC_target_dict.keys()
                    }

                

                for cl, genes in  source_dict.items():
                    set_i = set(genes)  # interactions in this cluster
                    #print(f"set {cl}", set_i)  
                    others = set().union(*[source_dict[j] for j in source_dict if j != cl])
                    #print("j", [RCC_source_sets[j] for j in RCC_source_sets.keys() if j != cl])
                    #print("others", others)
                    unique = set_i - others  # LRs only in this cluster
                    #print("unique", unique)
                    source_dict_unique[cl] = unique

            
                for cl, genes in  target_dict.items():
                    set_i = set(genes)  # interactions in this cluster
                    #print(f"set {cl}", set_i)  
                    others = set().union(*[target_dict[j] for j in target_dict if j != cl])
                    unique = set_i - others  # LRs only in this cluster
                    #print("unique", unique)
                    target_dict_unique[cl] = unique

                #print(source_dict)
                #print(target_dict)

                for cl, val in source_dict_unique.items():
                    source_dict_unique[cl] = {item.split("_", 1)[0] for item in source_dict_unique[cl]}

                for cl in source_dict_unique.keys():
                    all_interactions_list.append(source_dict_unique[cl].union(target_dict_unique[cl]))
                            
                #all_interactions_list = list(all_interactions_dict.values())

                print("all interactions", all_interactions_list)

    if opt2 == "a" or opt2 =="c" or opt2 == "d" or opt2 == "e":
        if opt == "set" or opt == "threshold":     
            print("all interactions", all_interactions_list)   
            return all_interactions_list, test_clust
        if opt == "unique":
            print("all interactions", all_interactions_list)
            return unique_per_cluster, test_clust
        
    if opt2 == "a2":
        if opt == "unique":
            return all_interactions_list, test_clust

    if opt2 == "b" or opt2 == "d2":
        if opt == "set" or opt == "threshold":        
            return all_interactions_source_list, all_interactions_target_list, test_clust
        if opt == "unique":
            return unique_per_cluster_source, unique_per_cluster_target, test_clust
    

In [ ]:
def get_GO_terms(opt = "filter"):

    go = GODag("go-basic.obo")

    anchors = {
        "Monocytes_Lymphocytes": [
            "GO:0002376", # immune system process
            "GO:0002520", # immune system development
            "GO:0006955", # immune response
            "GO:0030097", # hemopoiesis
            "GO:0002764", # immune response-regulating signaling pathway
            "GO:0002250", # adaptive immune response
            "GO:0006665" #sphingolipid  metabolic process
        ],
        "Tumor_Prog": [
            "GO:0007049", # cell cycle
            "GO:0008283", # cell proliferation
            "GO:0030154", # cell differentiation
            "GO:0001837", # epithelial to mesenchymal transition
            "GO:0001525", # angiogenesis
            "GO:0001666", # response to hypoxia,
            "GO:0002841", #regulation of T cell mediated immune response to tumor cell
            "GO:0002347", #response to tumor cell
            "GO:0002357", # defense response to tumor cell
            "GO:0071228", #cellular response to tumor cell,
            "GO:0008219", #cell death
            "GO:0006914", #autophagy
            "GO:0006281", #DNA repair
            "GO:0007165",  #signal transduction
            "GO:0002248", #connective tissue replacement involved in inflammatory response wound healing
            "GO:0042060", #wound healing
            "GO:1903747", #positive regulation of fibronectin production
            "GO:0006986", #response to unfolded protein
            "GO:0036503" #ERAD pathway
        ],
        "Kidney": [
            "GO:0072001", # renal system development
            "GO:0001822", # kidney development
            "GO:0072073", # kidney epithelial cell differentiation
            "GO:0072078", # nephron development
        ],
        "Endothelial": [
            "GO:0001944", # vasculature development
            "GO:0001525", # angiogenesis
            "GO:0001568", # blood vessel development
            "GO:0003018", # vascular process in circulatory system
        ],
        "Stromal_Bone": [
            "GO:0001501", # skeletal system development
            "GO:0001649", # osteoblast differentiation
            "GO:0030198", # extracellular matrix organization
            "GO:0043062", # extracellular structure organization
            "GO:0048659", #smooth muscle cell proliferation
            "GO:0003012", # muscle system process 
            "GO:0048762", # mesenchymal cell differentiation
            "GO:0007155", #cell adhesion
            "GO:0098645", #collagen network
            "GO:0005581", #collagen trimer
            "GO:0001503",#ossification
            "GO:0060612" #adipose tissue development
        ],
        "MFs": [
            "GO:0005126", # cytokine receptor binding
            "GO:0031723", #chemokine receptor binding: 
            "GO:0003823", #antigen binding
            "GO:0015036", #oxidoreductase activity
            "GO:0000155", #kinase activity
            "GO:0000121", #phosphatase activity
            "GO:0005520", #growth factor binding:
            "GO:0048018", #receptor ligand activity
            "GO:0000981", #DNA-binding transcription factor activity
            "GO:0005518", #collagen binding
            "GO:0046810", #extracellular matrix binding
            "GO:0034987", #immunoglobulin receptor binding
            "GO:0008201", #heparin binding:
            "GO:0005125", #cytokine activity
            "GO:0005260", #intracellularly ATP-gated chloride channel activity
            "GO:0005201", #extracellular matrix structural constituent
            "GO:0008191", #metalloendopeptidase activity
            "GO:0005546", #phosphatidylinositol-4,5-bisphosphate binding
            "GO:0004222", #cysteine-type endopeptidase activity	ECM degradation during remodeling
            "GO:0008237",	#metallopeptidase activity
            "GO:0019961" #interferon binding
        ]
    }


    def get_children(go_ids):
        terms = set()
        terms_children = set()
        for go_id in go_ids:
            if go_id in go:
                terms.add(go_id)
                terms.update(go[go_id].get_all_children())
                terms_children.update(go[go_id].get_all_children())
        return terms

    # collect all terms across categories
    allowed_terms = set()
    for category, go_ids in anchors.items():
        allowed_terms.update(get_children(go_ids))
    print(len(allowed_terms))

    if opt == "filter":
        def load_annotations_from_gaf(path):
            """Load a GAF file (can be gzipped). Returns:
            - gene_to_go: dict[gname] -> set(GO IDs)
            - go_to_genes: dict[go] -> set(gname)
            Uses DB_Object_Symbol (col 3, zero-based index 2) as gene/key and GO_ID (col 5, index 4).
            """
            df = pd.read_csv(path, sep='\t', comment='!', header=None, dtype=str)
            # ensure at least 5 columns
            df = df.loc[:, [2,4]].dropna()
            df.columns = ['gene','go']
            gene_to_go = df.groupby('gene')['go'].agg(lambda s: set(s)).to_dict()
            go_to_genes = df.groupby('go')['gene'].agg(lambda s: set(s)).to_dict()
            return gene_to_go, go_to_genes


        def count_term_size(go_to_genes):
            return {go: len(genes) for go, genes in go_to_genes.items()}

        gene_to_go, go_to_genes = load_annotations_from_gaf("goa_human.gaf")

        term_sizes = count_term_size(go_to_genes)
        filtered_terms = {go for go, n in term_sizes.items() if n <=400}

        allowed_terms = allowed_terms.intersection(go_to_genes.keys())
        print(len(allowed_terms))

        filtered_out_terms = ['cardiac muscle cell differentiation', 'lung vasculature development',
                      "antigen processing and presentation", "cell differentiation", "positive regulation of gene expression", 
                      #"positive regulation of cell population proliferation",
                      "inflammatory response", "signal transduction", 
                      "innate immune response", "immune response", "immune system process","adaptive immune response", 
                      "cell population proliferation", "lung epithelial cell differentiation", "microglial cell activation", 
                      "astrocyte activation involved in immune response", "microglia development", "complement activation", 
                      "complement activation, classical pathway", "complement activation, lectin pathway", 
                      "complement activation, alternative pathway", "complement activation, GZMK pathway", #
                      "complement receptor mediated signaling pathway", "neurogenesis", "microglial cell activation involved in immune response", 
                      'immune effector process', 'cardiac muscle cell myoblast differentiation', 'kidney development', 'ureteric bud development',
                      'retina vasculature development in camera-type eye', 'glial cell proliferation', 'inner ear auditory receptor cell differentiation',
                      'neuroendocrine cell differentiation', 'aorta development', #'glial cell differentiation', 
                      'coronary vasculature development', "neuron differentiation", "dopaminergic neuron differentiation", 
                      'endocardial cushion to mesenchymal transition', 'epithelial to mesenchymal transition involved in endocardial cushion formation', 
                      'angiogenesis involved in coronary vascular morphogenesis', 'cerebellar granule cell precursor proliferation', 
                      "neural precursor cell proliferation", 'negative regulation of blood-brain barrier permeability',
                      'Bergmann glial cell differentiation', 'type II pneumocyte differentiation', 'type I pneumocyte differentiation', "microglial cell differentiation", 
                      'Cajal-Retzius cell differentiation', "neuroblast proliferation", "T cell differentiation in thymus", "odontoblast differentiation",
                      "cardiac epithelial to mesenchymal transition", "cardiac muscle contraction", "central nervous system vasculogenesis", 
                      "calcineurin-mediated signaling", "calcium-mediated signaling", "apoptotic process", 'Sertoli cell differentiation', "cell adhesion",
                      "G protein-coupled receptor signaling pathway"]

    return allowed_terms, filtered_out_terms


In [ ]:
def print_go_mapping_stats(res, all_genes):

    mapped_genes = set()
    unmapped_genes = set(all_genes)

    for r in res:
        if "go" in r and r.get("entrezgene") is not None:
            symbol = r.get("symbol", r.get("query"))
            mapped_genes.add(symbol)
            if symbol in unmapped_genes:
                unmapped_genes.remove(symbol)

    print("GO Mapping")
    print(f"Total genes: {len(all_genes)}")
    print(f"Number of mapped genes: {len(mapped_genes)}")
    print(f"Number of unmapped genes: {len(unmapped_genes)}")
    print(f"Percentage mapped: {len(mapped_genes)/len(all_genes)*100:.2f}%")
    #print("\nMapped genes:", list(mapped_genes))
    print("Unmapped genes:", list(unmapped_genes))

    return mapped_genes, unmapped_genes


In [ ]:
#adapted from pycrosstalker
#prints source to ligand sankeys
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import seaborn as sns
import plotly.colors as pc
import plotly.graph_objects as go
from plotnine import *
from adjustText import adjust_text
from gprofiler import GProfiler
from sankeyflow import Sankey
import json

def plot_sankey_half(lrobj_tbl, target = None, ligand_cluster = None, receptor_cluster = None, plt_name = None, threshold = 50, tfflag = True):
    """
    This function selected genes sankey plot

     Parameters
    ----------
    lrobj_tbl :
        LRobject table with all data

    target :
        gene

    ligand_cluster :
        Ligand Clusters

    receptor_cluster :
        Receptor Clusters

    plt_name :
        plot title

    threshold :
        top_n n value
    
    Returns
    -------
    Python default plot

    """

    lrobj_tbl = lrobj_tbl[(lrobj_tbl['type_gene_A'] == "Ligand") & (lrobj_tbl['type_gene_B'] == "Receptor")]

    if target is not None:
        if len(target.split('|')) > 1:
            target_type = str(target.split('|')[1])
            if target_type == 'R':
                if lrobj_tbl['gene_B'].str.contains('\\|').any():
                    pass
                else:
                    target = target.split('|')[0]
                data = lrobj_tbl[lrobj_tbl['gene_B'] == target]
            elif target_type == 'L':
                if lrobj_tbl['gene_A'].str.contains('\\|').any():
                    pass
                else:
                    target = target.split('|')[0]
                data = lrobj_tbl[lrobj_tbl['gene_A'] == target]
        else:
            data = lrobj_tbl[lrobj_tbl['allpair'].str.contains(target)]
    else:
        data = lrobj_tbl

    
    if ligand_cluster is not None:
        data = data[data['source'].isin(ligand_cluster)]
    
    if receptor_cluster is not None:
        data = data[data['target'].isin(receptor_cluster)]


    color_palette = ['#00BFC4', '#FF3E3E']

    
    if len(data) >= 1:
        cat_cols = ['source', 'gene_A']
        value_cols = 'LRScore'
        data = data.loc[data['LRScore'].abs().nlargest(min(len(data), threshold)).index]
        title = plt_name

        gen_sankey(data, cat_cols, value_cols, title)
    
    else:
        print(f"Gene->{target} Not Found")
    

def gen_sankey(df, cat_cols=[], value_cols='', title='Sankey Diagram'):
    """
    Helper function to the function plot_sankey()

     Parameters
    ----------
    df :
        Dataframe

    cat_cols :
        Columns interested in the sankey plot

    value_cols :
        Sankey plot generated using connections based on this value_cols

    title :
        Title of Sankey plot
    
    Returns
    -------
    Nothing (plots Sankey plot)

    """

    # df['source'] += 'S'
    df['target'] += ' '
    
    labelList = []
    for catCol in cat_cols:
        labelListTemp =  list((df[catCol].values))
        labelList = labelList + labelListTemp    
        
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
    
    vmin = sourceTargetDf['count'].min()
    vmax = sourceTargetDf['count'].max()
    limit = max(abs(vmin), abs(vmax))
    vcenter = 0
    norm = mcolors.TwoSlopeNorm(vmin=-limit, vcenter=vcenter, vmax=limit)
    # norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    cmap = plt.get_cmap('RdBu_r')
    sourceTargetDf['hex_color'] = sourceTargetDf['count'].apply(lambda x: mcolors.to_hex(cmap(norm(x))))
    
    flows = []
    for i, row in sourceTargetDf.iterrows():
        flows.append((row['source'], row['target'], 1, {'color': row['hex_color']}))

    nodes = Sankey.infer_nodes(flows)
    nodes_new = []
    for level in nodes:
        level_new = []
        for node in level:
            node_new = node + [{'color' : 'black',
                                'label_pos':'center', 'label_opts': dict(fontsize=10, bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='white'))}]
            level_new.append(node_new)
        nodes_new.append(level_new)

    fig, ax = plt.subplots(figsize=(15, 10))
    s = Sankey(flows=flows,
               nodes=nodes_new,
               flow_color_mode_alpha=0.3,
               node_opts=dict(label_format='{label}'),
    )
    s.draw(ax=ax)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.01, shrink=0.5)
    cbar.set_label(value_cols, fontsize=12)

    ax.text(x=0.0, y=1.02, s="Source", fontsize=10)
    ax.text(x=1.0, y=1.02, s="Ligand", fontsize=10)

    ax.set_title(title)
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_GO(f):
    

    allowed_terms, filtered_out_terms = get_GO_terms()

    clusters = list(cluster_dict.values())

    all_interactions = f
    df_full = pd.DataFrame() 
    genes_bps_sep = {}
    top_terms_list = []
    top_terms_list_MF = []
    for i in range(len(all_interactions)):
        markers_tst = list(all_interactions[i])

        s = "_"
        clust_name = s.join(clusters[i]).replace("/", "_")
        print(clust_name)
        print(clusters[i])
        pre_genes = []
        for m in markers_tst:
            splitt = m.split("_")
            pre_genes.append(splitt)

        genes = []
        for xs in pre_genes:
            for x in xs:
                genes.append(x)
                
        genes = list(np.unique(genes))

        mg = mygene.MyGeneInfo()

        res = mg.querymany(
            genes,
            scopes="symbol",
            fields="entrezgene,ensembl.gene,go",
            species="human"
        )

        rows = []
        for r in res:
            geneid = r.get("entrezgene")
            genename = r.get("symbol", r["query"]) 
            if geneid is None or "go" not in r:
                continue
            gos = r["go"]
            if isinstance(gos, dict):
                for ns, terms in gos.items():  # ns = BP, MF, CC
                    if isinstance(terms, list):
                        for t in terms:
                            rows.append({"Gene": genename, "GeneID": geneid, "GOID": t["id"], "Namespace": ns, "GOName": t["term"]})
                    elif isinstance(terms, dict):
                        rows.append({"Gene": genename, "GeneID": geneid, "GOID": terms["id"], "Namespace": ns, "GOName": terms["term"]})


        df = pd.DataFrame(rows)
        if not df.empty:
            df = df[df["GOID"].isin(allowed_terms)]
            df = df[~df["GOName"].isin(filtered_out_terms)]
            #df = df[df["GOName"].str.contains("pathway")==False]
            df = df[df["GOName"].str.contains("cardiac")==False]
            df = df[df["GOName"].str.contains("lens")==False]
            #df = df[df["GOName"].str.contains("thymic")==False]
            df = df[df["GOName"].str.contains("neural")==False]
            df = df[df["GOName"].str.contains("neuron")==False]
            df = df[df["GOName"].str.contains("brain")==False]
            df = df[df["GOName"].str.contains("Langerhans")==False]
            genes_bps_tmp = df.groupby("GOName")["Gene"].apply(lambda x: list(set(x)))

        genes_bps_sep[i] = genes_bps_tmp
        df_full = pd.concat((df_full, df))

        if df.empty:
            top_terms_list.append(set())
            top_terms_list_MF.append(set())
            continue

        go_counts = df.groupby(["Namespace", "GOID", "GOName"]).count().rename(columns={"GeneID": "GeneCount"}).reset_index()

        print(pre_genes)
        unique_pairs = df[["Gene", "GOName"]].drop_duplicates()
        for gene, go in zip(unique_pairs["Gene"], unique_pairs["GOName"]):
            print(gene, "→", go)

        mapped_genes, unmapped_genes = print_go_mapping_stats(res, genes)
        
        for ns in ["BP", "MF", "CC"]:
        
            top_terms = go_counts[go_counts.Namespace==ns].sort_values("GeneCount", ascending=False)
            top_terms_plot = go_counts[go_counts.Namespace==ns].sort_values("GeneCount", ascending=False).head(30)
            
            plt.figure(figsize=(10,6))
            sns.barplot(
                data=top_terms_plot,
                y="GOName",
                x="GeneCount",
                palette="viridis"
            )
            if ns == "BP":
                top_terms_list.append(top_terms["GOName"])
            elif ns == "MF":
                top_terms_list_MF.append(top_terms["GOName"])

            plt.xlabel("Number of Genes")
            plt.ylabel(f"GO {ns} Terms")
            #plt.title(f"{clust_name}")
            plt.tight_layout()
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1a_common_LRs_cluster_all_LR_pairs_unchanged/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1b_common_LRs_cluster_separate_incoming_outgoing/incoming/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1b_common_LRs_cluster_separate_incoming_outgoing/outgoing/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform/incoming/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform/outgoing/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform/set/RBERVertex/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/RBConfig_1_25/set/c/{clusters[i][0]}_top_30_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/endgame/CPM_0_31/unique/c/{clusters[i][0]}_top_30_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\RBConfig_1_25\\threshold\\c\\{clusters[i][0]}_top_30_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\RBConfig_1_25\\set\\c\\{clusters[i][0]}_top_30_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform\\set\\RBERVertex\\unique\\{clusters[i][0]}_top_20_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\res025\\unique\\{clusters[i][0]}_top_20_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\res031\\unique_c\\{clusters[i][0]}_top_20_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\res031\\unique_d\\{clusters[i][0]}_top_20_BPs_GO.png")
            plt.show()
    return df_full, top_terms_list, top_terms_list_MF


In [ ]:
def plot_unique_bp_sankeys(f, clusters_cleaned, df_full, top_terms_list, intra_inter, which_os = "linux", cluster_dict = cluster_dict, overview = None, experimental = None):    

    def flatten(xss):
        return [x for xs in xss for x in xs]
    
    all_interactions = f
    clusters = list(cluster_dict.values())
    #print(df_full)
    df_full_bp = df_full[df_full["Namespace"] == "BP"]
    df_full_bp = df_full_bp[["Gene", "GOName"]]
    
    #df_full_bp.pivot(index="GOName", columns="Gene")
    genes_bps = df_full_bp.groupby("GOName")["Gene"].apply(lambda x: list(set(x)))

    #get bp terms that only appear in one single cluster
    unique_per_cluster = []

    #the <2 not important here, just didnt delete bc i copied it from the unique genes function
    for i, cluster in enumerate(top_terms_list):
        if len(clusters[i]) < 1:
            unique_per_cluster.append([])
            continue
        set_i = set(cluster)  # interactions in this cluster
        others = set().union(*[top_terms_list[j]for j in range(len(top_terms_list)) if j != i])
        unique = set_i - others  # LRs only in this cluster
        unique_per_cluster.append(unique)

    for idx, uniq in enumerate(unique_per_cluster):
        cluster_key = list(cluster_dict.keys())[idx]
        print(f"{cluster_key}: {uniq}")

    genes_bps_sub = genes_bps[genes_bps.index.isin(flatten(unique_per_cluster))]

    method= "d"

    interactions_in_bp = {}
    interaction_genes_only = {}


    for i, lr_sets in enumerate(all_interactions):
        if not (lr_sets):
            continue
        cluster_hits = []
        interaction_genes = []
        #print(unique_per_cluster[i])
        for j in unique_per_cluster[i]:
            
            bps_of_interest = [j]

            #print(bps_of_interest)
            genes_bps_sub = genes_bps[genes_bps.index.isin(bps_of_interest)]
            #print(genes_bps_sub)
            
            
            for interaction in lr_sets:
                # split lr into ligand and receptor
                if "_" in interaction:
                    genes = interaction.split("_")
                else:
                    genes = [interaction]
                #print(genes)
                for bp, genes_in_bp in genes_bps_sub.items():
                    if not isinstance(genes_in_bp, set):
                        genes_in_bp = set(genes_in_bp)

                    #print(genes_in_bp)
                    # are both l and r in this biological process
                    matching = [g for g in genes if g in genes_in_bp]
                    #matching = [(g, "ligand" if idx==0 else "receptor") 
                    #     for idx, g in enumerate(genes) if g in genes_in_bp]
                    #print(matching)
                    # if both ligand and receptor are in this BP then keep
                    if method == "d":
                        if len(matching) >= 1:  
                            cluster_hits.append((clusters[i], interaction, bp, matching))
                            interaction_genes.append(interaction)

                    else:
                        if len(matching) >= 2:  
                            cluster_hits.append((clusters[i], interaction, bp, matching))
                            interaction_genes.append(interaction)

        interactions_in_bp[i] = cluster_hits
        interaction_genes_only[i] = interaction_genes


    for i, val in interaction_genes_only.items():
        print(len(set(val)))

    import contextvars
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri, default_converter
    from rpy2.robjects.conversion import localconverter, set_conversion

    if which_os == "windows11":

        set_conversion(default_converter)
        ctx = contextvars.copy_context()

        def load_diff_table(path: str):
            def _read():
                readRDS = ro.r['readRDS']
                df = readRDS(path)
                tables = df.slots['tables']

                diff_table = tables[2]
                with localconverter(default_converter + pandas2ri.converter):
                    return ro.conversion.rpy2py(diff_table)

            return ctx.run(_read)


    if which_os == "linux":

        readRDS = ro.r['readRDS']
        df_RCC = readRDS("/home/larissa/Documents/Masterarbeit/RCC_results/crosstalker/no_subset_glom_removed/LR_data_final.Rds")
        df_MF = readRDS("/home/larissa/Documents/Masterarbeit/MF_results/crosstalker/all_celltypes/pval0_05/LR_data_final.Rds")

        
        tables_RCC = df_RCC.slots["tables"]
        diff_table_RCC = tables_RCC[2]
        RCC_diff_df = pandas2ri.rpy2py(diff_table_RCC)

        tables_MF = df_MF.slots["tables"]
        diff_table_MF = tables_MF[2]
        MF_diff_df = pandas2ri.rpy2py(diff_table_MF)

    elif which_os == "windows":
        readRDS = ro.r['readRDS']
        df_RCC = readRDS("/home/larissa/Documents/Masterarbeit/RCC_results/crosstalker/no_subset_glom_removed/LR_data_final.Rds")
        df_MF = readRDS("/home/larissa/Documents/Masterarbeit/MF_results/crosstalker/all_celltypes/pval0_05/LR_data_final.Rds")

        

        tables_RCC = df_RCC.slots["tables"]
        diff_table_RCC = tables_RCC[2]
        RCC_diff_df = pandas2ri.rpy2py(diff_table_RCC)

        tables_MF = df_MF.slots["tables"]

        diff_table_MF = tables_MF[2]
        MF_diff_df = pandas2ri.rpy2py(diff_table_MF)

    elif which_os == "windows11":
        df_RCC = "C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\RCC_results\\crosstalker\\final_filtering\\LR_data_final.Rds"
        df_MF = "C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\MF_results\\crosstalker\\final_filtering\\LR_data_final.Rds"


        RCC_diff_df = load_diff_table(df_RCC)
        MF_diff_df = load_diff_table(df_MF)


    RCC_diff_df["gene_A"] = RCC_diff_df["gene_A"].apply([lambda x: x.removesuffix("|L")])
    RCC_diff_df["gene_B"] = RCC_diff_df["gene_B"].apply([lambda x: x.removesuffix("|R")])
    RCC_diff_df["lr_pair"] = RCC_diff_df["gene_A"] + "_" + RCC_diff_df["gene_B"]
    RCC_diff_df = RCC_diff_df[(~RCC_diff_df["source"].isin(["Glomerular endothelium", "Podocytes", "OM Type A-ICs", "Type B-IC","Type A-ICs", "TAL of LOH"])) & (~RCC_diff_df["target"].isin(["Glomerular endothelium", "Podocytes", "OM Type A-ICs", "Type B-IC", "Type A-ICs", "TAL of LOH"]))]

    MF_diff_df["gene_A"] = MF_diff_df["gene_A"].apply([lambda x: x.removesuffix("|L")])
    MF_diff_df["gene_B"] = MF_diff_df["gene_B"].apply([lambda x: x.removesuffix("|R")])
    MF_diff_df["lr_pair"] = MF_diff_df["gene_A"] + "_" + MF_diff_df["gene_B"]
    MF_diff_df = MF_diff_df[(~MF_diff_df["source"].isin(["Fibroblasts"])) & (~MF_diff_df["target"].isin(["Fibroblasts"]))]

    #if not big_df.empty:
    #    interaction_genes_only = genes_only
    #print(interaction_genes_only)
    RCC_lrscores = pd.DataFrame()
    MF_lrscores = pd.DataFrame()
    RCC_df_list = []
    MF_df_list  = []

    for i in range(len(clusters_cleaned)):

        RCC_diff_df_filtered = RCC_diff_df[RCC_diff_df["source"].isin(clusters_cleaned[i]) | RCC_diff_df["target"].isin(clusters_cleaned[i])]
        MF_diff_df_filtered = MF_diff_df[MF_diff_df["source"].isin(clusters_cleaned[i]) | MF_diff_df["target"].isin(clusters_cleaned[i])]

        if intra_inter == "inter":

            if i in interaction_genes_only:
                #print("interaction_genes", interaction_genes_only)
                print(i)
                for gene in np.unique(interaction_genes_only[i]):
                    print(gene)
                    #RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["gene_B"] == gene)| (RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(RCC_diff_df_filtered_gene) > 0:
                        RCC_df_list.append(RCC_diff_df_filtered_gene)
                        RCC_lrscores = pd.concat((RCC_lrscores, RCC_diff_df_filtered_gene))

                    #MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["gene_A"] == gene) | (MF_diff_df_filtered["gene_B"] == gene) | (MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(MF_diff_df_filtered_gene) > 0:
                        MF_df_list.append(MF_diff_df_filtered_gene)
                        MF_lrscores = pd.concat((MF_lrscores, MF_diff_df_filtered_gene))

        elif intra_inter == "intra":
           
           if i in interaction_genes_only:
                #print("interaction_genes", interaction_genes_only)
                print(i)
                for gene in np.unique(interaction_genes_only[i]):
                    print(gene)
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["gene_B"] == gene)| (RCC_diff_df_filtered["lr_pair"] == gene)]
                    #RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(RCC_diff_df_filtered_gene) > 0:
                        RCC_df_list.append(RCC_diff_df_filtered_gene)
                        RCC_lrscores = pd.concat((RCC_lrscores, RCC_diff_df_filtered_gene))

                    #MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["gene_A"] == gene) | (MF_diff_df_filtered["gene_B"] == gene) | (MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(MF_diff_df_filtered_gene) > 0:
                        MF_df_list.append(MF_diff_df_filtered_gene)
                        MF_lrscores = pd.concat((MF_lrscores, MF_diff_df_filtered_gene))


    #print("df list", RCC_df_list)

    cluster_dict = {
        k: [v.removeprefix("RCC_").removeprefix("MF_") for v in vals]
        for k, vals in cluster_dict.items()
    }

    mapping = {
        item: cluster
        for cluster, items in cluster_dict.items()
        for item in items
    }

    grouped_list_RCC = []
    grouped_list_MF = []

    for df in RCC_df_list:
        df["cluster_t"] = df["target"].map(mapping)
        df["cluster_s"] = df["source"].map(mapping)

        grouped = df.groupby(["cluster_s", "cluster_t", "lr_pair"], as_index=False).agg({
        "LRScore": "mean"
        #"source": "first",    # or " ".join, list, etc.
        #"lr_pair": "first"  # repeat for other cols
        })
        grouped_list_RCC.append(grouped)

    #print(grouped_list_RCC)
    final_df_RCC = pd.concat(grouped_list_RCC, ignore_index=True)

    for df in MF_df_list:
        df["cluster_t"] = df["target"].map(mapping)
        df["cluster_s"] = df["source"].map(mapping)

        #grouped = df.groupby("cluster")["LRScore"].mean().reset_index()
        grouped = df.groupby(["cluster_s", "cluster_t", "lr_pair"], as_index=False).agg({
        "LRScore": "mean"
        #"source": "first",    # or " ".join, list, etc.
        #"lr_pair": "first"  # repeat for other cols
        })
        grouped_list_MF.append(grouped)

    final_df_MF = pd.concat(grouped_list_MF, ignore_index=True)

    RCC_final = final_df_RCC.rename(columns={"LRScore": "LRScore_RCC"})
    MF_final = final_df_MF.rename(columns={"LRScore": "LRScore_MF"})

    merged = pd.merge(
        RCC_final,
        MF_final,
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"   # only keep common cluster-target combos
    )


    merged["direction_RCC"] = merged["LRScore_RCC"].apply(lambda x: "pos" if x > 0 else "neg")
    merged["direction_MF"] = merged["LRScore_MF"].apply(lambda x: "pos" if x > 0 else "neg")

    merged["same_direction"] = merged["direction_RCC"] == merged["direction_MF"]

    subsetmerged = merged[merged["same_direction"] == True].drop_duplicates()

    if intra_inter == "intra":
                subsetmerged = subsetmerged[(subsetmerged["cluster_s"] == subsetmerged["cluster_t"])]

    elif intra_inter == "inter":
                subsetmerged = subsetmerged[(subsetmerged["cluster_s"] != subsetmerged["cluster_t"])]

    trend = merged.groupby("lr_pair").agg(
        mean_RCC=("LRScore_RCC", "mean"),
        mean_MF=("LRScore_MF", "mean")
    ).reset_index()

    trend["trend_RCC"] = trend["mean_RCC"].apply(lambda x: "pos" if x > 0 else ("neg" if x < 0 else "zero"))
    trend["trend_MF"]  = trend["mean_MF"].apply(lambda x: "pos" if x > 0 else ("neg" if x < 0 else "zero"))

    trend["same_overall_trend"] = trend["trend_RCC"] == trend["trend_MF"]
    trend_true = trend[trend["same_overall_trend"] == True]

    #mapped which cell types belong to which clusters and if they're the source or target cell type
    cluster_dict = {
        k: [v.removeprefix("RCC_").removeprefix("MF_") for v in vals]
        for k, vals in cluster_dict.items()
    }

    mapping = {
        item: cluster
        for cluster, items in cluster_dict.items()
        for item in items
    }

    grouped_list_RCC = []
    grouped_list_MF = []
    # Apply to each dataframe in RCC_df_list

    RCC_split = RCC_diff_df.copy()
    RCC_split["cluster_t"] = RCC_split["target"].map(mapping)
    RCC_split["cluster_s"] = RCC_split["source"].map(mapping)

    MF_split = MF_diff_df.copy()
    MF_split["cluster_t"] = MF_split["target"].map(mapping)
    MF_split["cluster_s"] = MF_split["source"].map(mapping)


    if intra_inter == "intra":
    #            RCC_split = RCC_split[(RCC_split["cluster_s"] == RCC_split["cluster_t"])]
    #            print(RCC_split)
    #            MF_split = MF_split[(MF_split["cluster_s"] == MF_split["cluster_t"])]  
    #
                RCC_split = RCC_split[(RCC_split["source"] != RCC_split["target"])]
                MF_split = MF_split[(MF_split["source"] != MF_split["target"])]  
    #elif intra_inter == "inter":
    #            RCC_split = RCC_split[(RCC_split["cluster_s"] != RCC_split["cluster_t"])]
    #            MF_split = MF_split[(MF_split["cluster_s"] != MF_split["cluster_t"])]  
                
    subsetmerged_RCC = RCC_split.merge(
        subsetmerged[["cluster_s", "cluster_t", "lr_pair"]],
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"
    )

    subsetmerged_MF = MF_split.merge(
        subsetmerged[["cluster_s", "cluster_t", "lr_pair"]],
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"
    )

    subsetmerged_RCC['gene_A'] = subsetmerged_RCC['gene_A'].astype(str) + '|L'
    subsetmerged_RCC['gene_B'] = subsetmerged_RCC['gene_B'].astype(str) + '|R'

    subsetmerged_MF['gene_A'] = subsetmerged_MF['gene_A'].astype(str) + '|L'
    subsetmerged_MF['gene_B'] = subsetmerged_MF['gene_B'].astype(str) + '|R'


    if overview ==  "yes":
        overview_RCC = subsetmerged_RCC
        overview_RCC["source"] = subsetmerged_RCC["cluster_s"]
        overview_RCC["target"] = subsetmerged_RCC["cluster_t"] 

        overview_MF = subsetmerged_MF
        overview_MF["source"] = subsetmerged_MF["cluster_s"]
        overview_MF["target"] = subsetmerged_MF["cluster_t"]      

    #if experimental == "yes":
                # Keep only Ligand-Receptor edges
    #            lrobj_tbl = subsetmerged_RCC
                # Incoming: source → ligand → receptor → target
    #            incoming = lrobj_tbl[['source', 'gene_A', 'gene_B', 'target', 'LRScore', "cluster_t"]].copy()

                # Outgoing: target → downstream ligand#
    #            outgoing = lrobj_tbl[['source', 'gene_A', 'LRScore']].copy()
    #            outgoing['downstream_l'] = outgoing['gene_A'] + "|DL"
    #            outgoing.rename(columns={'source': 'target', 'LRScore': 'LRScore_down'}, inplace=True)

                # Merge incoming with outgoing once
    #            merged_lr = incoming.merge(
    #                outgoing[['target', 'downstream_l', 'LRScore_down']],
    #                on='target',
    #                how='left'
    #            )

    for name, i in cluster_dict.items():
        #print(subset_RCC)
        #subsetmerged_RCC = subsetmerged_RCC[subsetmerged_RCC["lr_pair"].isin(interaction_genes_only[i])] 
        subsetmerged_RCC_s = subsetmerged_RCC[subsetmerged_RCC["source"].isin(i)] 
        subsetmerged_RCC_t = subsetmerged_RCC[subsetmerged_RCC["target"].isin(i)]

        #subsetmerged_MF = subsetmerged_MF[subsetmerged_MF["lr_pair"].isin(interaction_genes_only[i])] 
        subsetmerged_MF_s = subsetmerged_MF[subsetmerged_MF["source"].isin(i)]
        subsetmerged_MF_t = subsetmerged_MF[subsetmerged_MF["target"].isin(i)]

        if overview == "yes":
            
            subsetmerged_RCC_s = overview_RCC[overview_RCC["source"] == name] 
            subsetmerged_RCC_t = overview_RCC[overview_RCC["target"] == name]
        
            subsetmerged_MF_s = overview_MF[overview_MF["source"] == name]
            subsetmerged_MF_t = overview_MF[overview_MF["target"] == name]
             


        if intra_inter == "intra":
            print(name)
            print("RCC")
            pycrosstalker.pl.plot_sankey(subsetmerged_RCC_s,threshold = 50, plt_name = None)
            print("MF")
            pycrosstalker.pl.plot_sankey(subsetmerged_MF_s, threshold=50, plt_name =None)

        elif intra_inter == "inter":
            if experimental == "yes":
       
                #df_cluster = merged_lr[merged_lr['cluster_t'] == name]
                print(name)
                print("RCC")
                plot_sankey(subsetmerged_RCC, target_cluster = name, threshold =50, plt_name = None)
                #plot_sankey(subsetmerged_RCC_t,threshold = 100, plt_name = f"RCC  {name}")
                #print("MF")
                #plot_sankey(subsetmerged_MF_s, threshold=100, plt_name = f"MF {name}")
                #plot_sankey(subsetmerged_MF_t, threshold=100, plt_name = f"MF {name}")
            else:
                print(name)
                print("RCC")
                plot_sankey_half(subsetmerged_RCC_s, threshold =50, plt_name = None)
                pycrosstalker.pl.plot_sankey(subsetmerged_RCC_t,threshold = 50, plt_name =None)
                print("MF")
                plot_sankey_half(subsetmerged_MF_s, threshold=50, plt_name = None)
                pycrosstalker.pl.plot_sankey(subsetmerged_MF_t, threshold=50, plt_name =None)


    return subsetmerged, subsetmerged_RCC, subsetmerged_MF

In [ ]:
def plot_unique_mf_sankeys(f, clusters_cleaned, df_full, top_terms_list_MF, intra_inter, which_os = "linux", cluster_dict = cluster_dict, overview = None, experimental = None):    

    def flatten(xss):
        return [x for xs in xss for x in xs]
    
    all_interactions = f
    clusters = list(cluster_dict.values())
    df_full_mf = df_full[df_full["Namespace"] == "MF"]
    df_full_mf = df_full_mf[["Gene", "GOName"]]
    #print(df_full_mf)
    #print(top_terms_list_MF)
    #df_full_bp.pivot(index="GOName", columns="Gene")
    genes_mfs = df_full_mf.groupby("GOName")["Gene"].apply(lambda x: list(set(x)))

   
    unique_per_cluster = []

    for i, cluster in enumerate(top_terms_list_MF):
        if len(clusters[i]) < 1:
            #unique_per_cluster.append(set())
            continue
        set_i = set(cluster)  # interactions in this cluster
        others = set().union(*[top_terms_list_MF[j]for j in range(len(top_terms_list_MF)) if j != i])
        unique = set_i - others  # LRs only in this cluster
        unique_per_cluster.append(unique)


    for idx, uniq in enumerate(unique_per_cluster):
        cluster_key = list(cluster_dict.keys())[idx]
        print(f"{cluster_key}: {uniq}")

    genes_mfs_sub = genes_mfs[genes_mfs.index.isin(flatten(unique_per_cluster))]

    method= "d"

    interactions_in_mf = {}
    interaction_genes_only_mf = {}


    for i, lr_sets in enumerate(all_interactions):
        if not (lr_sets):
            continue
        cluster_hits = []
        interaction_genes = []
        #print(unique_per_cluster[i])
        for j in unique_per_cluster[i]:
            
            mfs_of_interest = [j]

            #print(bps_of_interest)
            genes_mfs_sub = genes_mfs[genes_mfs.index.isin(mfs_of_interest)]
            #print(genes_bps_sub)
            
            
            for interaction in lr_sets:
                # split lr into ligand and receptor
                if "_" in interaction:
                    genes = interaction.split("_")
                else:
                    genes = [interaction]
                #print(genes)
                for mf, genes_in_mf in genes_mfs_sub.items():
                    if not isinstance(genes_in_mf, set):
                        genes_in_mf = set(genes_in_mf)

                    #print(genes_in_bp)
                    # are both l and r in this biological process
                    matching = [g for g in genes if g in genes_in_mf]
                    #matching = [(g, "ligand" if idx==0 else "receptor") 
                    #     for idx, g in enumerate(genes) if g in genes_in_bp]
                    #print(matching)
                    # if both ligand and receptor are in this BP then keep
                    if method == "d":
                        if len(matching) >= 1:  
                            cluster_hits.append((clusters[i], interaction, mf, matching))
                            interaction_genes.append(interaction)


                    else:
                        if len(matching) >= 2:  
                            cluster_hits.append((clusters[i], interaction, mf, matching))
                            interaction_genes.append(interaction)

        interactions_in_mf[i] = cluster_hits
        interaction_genes_only_mf[i] = interaction_genes

        
    print("inter action genes", interaction_genes_only_mf)
    for i, val in interaction_genes_only_mf.items():
        print(len(set(val)))

    import contextvars
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri, default_converter
    from rpy2.robjects.conversion import localconverter, set_conversion

    if which_os == "windows11":
    
        set_conversion(default_converter)

        ctx = contextvars.copy_context()

        def load_diff_table(path: str):
            def _read():
                readRDS = ro.r['readRDS']
                df = readRDS(path)
                tables = df.slots['tables']
                diff_table = tables[2]
                with localconverter(default_converter + pandas2ri.converter):
                    return ro.conversion.rpy2py(diff_table)
     
            return ctx.run(_read)

    if which_os == "linux":

        readRDS = ro.r['readRDS']
        df_RCC = readRDS("/home/larissa/Documents/Masterarbeit/RCC_results/crosstalker/no_subset_glom_removed/LR_data_final.Rds")
        df_MF = readRDS("/home/larissa/Documents/Masterarbeit/MF_results/crosstalker/all_celltypes/pval0_05/LR_data_final.Rds")

        
        tables_RCC = df_RCC.slots["tables"]
        diff_table_RCC = tables_RCC[2]
        RCC_diff_df = pandas2ri.rpy2py(diff_table_RCC)

        tables_MF = df_MF.slots["tables"]
        diff_table_MF = tables_MF[2]
        MF_diff_df = pandas2ri.rpy2py(diff_table_MF)

    elif which_os == "windows":
        readRDS = ro.r['readRDS']
        df_RCC = readRDS("/home/larissa/Documents/Masterarbeit/RCC_results/crosstalker/no_subset_glom_removed/LR_data_final.Rds")
        df_MF = readRDS("/home/larissa/Documents/Masterarbeit/MF_results/crosstalker/all_celltypes/pval0_05/LR_data_final.Rds")

        

        tables_RCC = df_RCC.slots["tables"]
        diff_table_RCC = tables_RCC[2]
        RCC_diff_df = pandas2ri.rpy2py(diff_table_RCC)

        tables_MF = df_MF.slots["tables"]
        diff_table_MF = tables_MF[2]
        MF_diff_df = pandas2ri.rpy2py(diff_table_MF)

    elif which_os == "windows11":
        df_RCC = "C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\RCC_results\\crosstalker\\final_filtering\\LR_data_final.Rds"
        df_MF = "C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\MF_results\\crosstalker\\final_filtering\\LR_data_final.Rds"

        RCC_diff_df = load_diff_table(df_RCC)
        MF_diff_df = load_diff_table(df_MF)


    RCC_diff_df["gene_A"] = RCC_diff_df["gene_A"].apply([lambda x: x.removesuffix("|L")])
    RCC_diff_df["gene_B"] = RCC_diff_df["gene_B"].apply([lambda x: x.removesuffix("|R")])
    RCC_diff_df["lr_pair"] = RCC_diff_df["gene_A"] + "_" + RCC_diff_df["gene_B"]
    RCC_diff_df = RCC_diff_df[(~RCC_diff_df["source"].isin(["Glomerular endothelium", "Podocytes", "OM Type A-ICs", "Type B-IC","Type A-ICs", "TAL of LOH"])) & (~RCC_diff_df["target"].isin(["Glomerular endothelium", "Podocytes", "OM Type A-ICs", "Type B-IC", "Type A-ICs", "TAL of LOH"]))]

    MF_diff_df["gene_A"] = MF_diff_df["gene_A"].apply([lambda x: x.removesuffix("|L")])
    MF_diff_df["gene_B"] = MF_diff_df["gene_B"].apply([lambda x: x.removesuffix("|R")])
    MF_diff_df["lr_pair"] = MF_diff_df["gene_A"] + "_" + MF_diff_df["gene_B"]
    MF_diff_df = MF_diff_df[(~MF_diff_df["source"].isin(["Fibroblasts"])) & (~MF_diff_df["target"].isin(["Fibroblasts"]))]

    #if not big_df.empty:
    #    interaction_genes_only = genes_only
    #print(interaction_genes_only)
    RCC_lrscores = pd.DataFrame()
    MF_lrscores = pd.DataFrame()
    RCC_df_list = []
    MF_df_list  = []

    for i in range(len(clusters_cleaned)):

        RCC_diff_df_filtered = RCC_diff_df[RCC_diff_df["source"].isin(clusters_cleaned[i]) | RCC_diff_df["target"].isin(clusters_cleaned[i])]
        MF_diff_df_filtered = MF_diff_df[MF_diff_df["source"].isin(clusters_cleaned[i]) | MF_diff_df["target"].isin(clusters_cleaned[i])]

        if intra_inter == "inter":

            if i in interaction_genes_only_mf:
                print("interaction_genes", interaction_genes_only_mf)
                print(i)
                for gene in np.unique(interaction_genes_only_mf[i]):
                    print(gene)
                    #RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["gene_B"] == gene)| (RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(RCC_diff_df_filtered_gene) > 0:
                        RCC_df_list.append(RCC_diff_df_filtered_gene)
                        RCC_lrscores = pd.concat((RCC_lrscores, RCC_diff_df_filtered_gene))

                    #MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["gene_A"] == gene) | (MF_diff_df_filtered["gene_B"] == gene) | (MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(MF_diff_df_filtered_gene) > 0:
                        MF_df_list.append(MF_diff_df_filtered_gene)
                        MF_lrscores = pd.concat((MF_lrscores, MF_diff_df_filtered_gene))
        elif intra_inter == "intra":
           
           if i in interaction_genes_only_mf:
                print("interaction_genes", interaction_genes_only_mf)
                print(i)
                for gene in np.unique(interaction_genes_only_mf[i]):
                    print(gene)
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["gene_B"] == gene)| (RCC_diff_df_filtered["lr_pair"] == gene)]
                    #RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(RCC_diff_df_filtered_gene) > 0:
                        RCC_df_list.append(RCC_diff_df_filtered_gene)
                        RCC_lrscores = pd.concat((RCC_lrscores, RCC_diff_df_filtered_gene))

                    MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["gene_A"] == gene) | (MF_diff_df_filtered["gene_B"] == gene) | (MF_diff_df_filtered["lr_pair"] == gene)]
                    #MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(MF_diff_df_filtered_gene) > 0:
                        MF_df_list.append(MF_diff_df_filtered_gene)
                        MF_lrscores = pd.concat((MF_lrscores, MF_diff_df_filtered_gene))

    cluster_dict = {
        k: [v.removeprefix("RCC_").removeprefix("MF_") for v in vals]
        for k, vals in cluster_dict.items()
    }
    #mapping column to specified clusters
    mapping = {
        item: cluster
        for cluster, items in cluster_dict.items()
        for item in items
    }

    grouped_list_RCC = []
    grouped_list_MF = []

    for df in RCC_df_list:

        df["cluster_t"] = df["target"].map(mapping)
        df["cluster_s"] = df["source"].map(mapping)

        grouped = df.groupby(["cluster_s", "cluster_t", "lr_pair"], as_index=False).agg({
        "LRScore": "mean"
        #"source": "first",    # or " ".join, list, etc.
        #"lr_pair": "first"  # repeat for other cols
        })
        grouped_list_RCC.append(grouped)

    #print(grouped_list_RCC)
    final_df_RCC = pd.concat(grouped_list_RCC, ignore_index=True)

    for df in MF_df_list:
        df["cluster_t"] = df["target"].map(mapping)
        df["cluster_s"] = df["source"].map(mapping)

        #grouped = df.groupby("cluster")["LRScore"].mean().reset_index()
        grouped = df.groupby(["cluster_s", "cluster_t", "lr_pair"], as_index=False).agg({
        "LRScore": "mean"
        #"source": "first",    # or " ".join, list, etc.
        #"lr_pair": "first"  # repeat for other cols
        })
        grouped_list_MF.append(grouped)

    final_df_MF = pd.concat(grouped_list_MF, ignore_index=True)

    RCC_final = final_df_RCC.rename(columns={"LRScore": "LRScore_RCC"})
    MF_final = final_df_MF.rename(columns={"LRScore": "LRScore_MF"})

    merged = pd.merge(
        RCC_final,
        MF_final,
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"   # only keep common cluster-target combos
    )


    merged["direction_RCC"] = merged["LRScore_RCC"].apply(lambda x: "pos" if x > 0 else "neg")
    merged["direction_MF"] = merged["LRScore_MF"].apply(lambda x: "pos" if x > 0 else "neg")

    merged["same_direction"] = merged["direction_RCC"] == merged["direction_MF"]

    subsetmerged = merged[merged["same_direction"] == True].drop_duplicates()

    if intra_inter == "intra":
                subsetmerged = subsetmerged[(subsetmerged["cluster_s"] == subsetmerged["cluster_t"])]

    elif intra_inter == "inter":
                subsetmerged = subsetmerged[(subsetmerged["cluster_s"] != subsetmerged["cluster_t"])]

    trend = merged.groupby("lr_pair").agg(
        mean_RCC=("LRScore_RCC", "mean"),
        mean_MF=("LRScore_MF", "mean")
    ).reset_index()


    trend["trend_RCC"] = trend["mean_RCC"].apply(lambda x: "pos" if x > 0 else ("neg" if x < 0 else "zero"))
    trend["trend_MF"]  = trend["mean_MF"].apply(lambda x: "pos" if x > 0 else ("neg" if x < 0 else "zero"))

    trend["same_overall_trend"] = trend["trend_RCC"] == trend["trend_MF"]
    trend_true = trend[trend["same_overall_trend"] == True]

    #mapped which cell types belong to which clusters and if they're the source or target cell type
    cluster_dict = {
        k: [v.removeprefix("RCC_").removeprefix("MF_") for v in vals]
        for k, vals in cluster_dict.items()
    }

    mapping = {
        item: cluster
        for cluster, items in cluster_dict.items()
        for item in items
    }

    grouped_list_RCC = []
    grouped_list_MF = []


    RCC_split = RCC_diff_df.copy()
    RCC_split["cluster_t"] = RCC_split["target"].map(mapping)
    RCC_split["cluster_s"] = RCC_split["source"].map(mapping)

    MF_split = MF_diff_df.copy()
    MF_split["cluster_t"] = MF_split["target"].map(mapping)
    MF_split["cluster_s"] = MF_split["source"].map(mapping)


    if intra_inter == "intra":
    #            RCC_split = RCC_split[(RCC_split["cluster_s"] == RCC_split["cluster_t"])]
    #            print(RCC_split)
    #            MF_split = MF_split[(MF_split["cluster_s"] == MF_split["cluster_t"])]  
    #
                RCC_split = RCC_split[(RCC_split["source"] != RCC_split["target"])]
                MF_split = MF_split[(MF_split["source"] != MF_split["target"])]  
    #elif intra_inter == "inter":
    #            RCC_split = RCC_split[(RCC_split["cluster_s"] != RCC_split["cluster_t"])]
    #            MF_split = MF_split[(MF_split["cluster_s"] != MF_split["cluster_t"])]  
                
    subsetmerged_RCC = RCC_split.merge(
        subsetmerged[["cluster_s", "cluster_t", "lr_pair"]],
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"
    )

    subsetmerged_MF = MF_split.merge(
        subsetmerged[["cluster_s", "cluster_t", "lr_pair"]],
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"
    )

    subsetmerged_RCC['gene_A'] = subsetmerged_RCC['gene_A'].astype(str) + '|L'
    subsetmerged_RCC['gene_B'] = subsetmerged_RCC['gene_B'].astype(str) + '|R'

    subsetmerged_MF['gene_A'] = subsetmerged_MF['gene_A'].astype(str) + '|L'
    subsetmerged_MF['gene_B'] = subsetmerged_MF['gene_B'].astype(str) + '|R'


    if overview ==  "yes":
        overview_RCC = subsetmerged_RCC
        overview_RCC["source"] = subsetmerged_RCC["cluster_s"]
        overview_RCC["target"] = subsetmerged_RCC["cluster_t"] 

        overview_MF = subsetmerged_MF
        overview_MF["source"] = subsetmerged_MF["cluster_s"]
        overview_MF["target"] = subsetmerged_MF["cluster_t"]      

    #if experimental == "yes":
                # Keep only Ligand-Receptor edges
    #            lrobj_tbl = subsetmerged_RCC
                # Incoming: source → ligand → receptor → target
    #            incoming = lrobj_tbl[['source', 'gene_A', 'gene_B', 'target', 'LRScore', "cluster_t"]].copy()

                # Outgoing: target → downstream ligand#
    #            outgoing = lrobj_tbl[['source', 'gene_A', 'LRScore']].copy()
    #            outgoing['downstream_l'] = outgoing['gene_A'] + "|DL"
    #            outgoing.rename(columns={'source': 'target', 'LRScore': 'LRScore_down'}, inplace=True)

                # Merge incoming with outgoing once
    #            merged_lr = incoming.merge(
    #                outgoing[['target', 'downstream_l', 'LRScore_down']],
    #                on='target',
    #                how='left'
    #            )

    for name, i in cluster_dict.items():
        #print(subset_RCC)
        subsetmerged_RCC_s = subsetmerged_RCC[subsetmerged_RCC["source"].isin(i)] 
        subsetmerged_RCC_t = subsetmerged_RCC[subsetmerged_RCC["target"].isin(i)]

        subsetmerged_MF_s = subsetmerged_MF[subsetmerged_MF["source"].isin(i)]
        subsetmerged_MF_t = subsetmerged_MF[subsetmerged_MF["target"].isin(i)]

        if overview == "yes":
            
            subsetmerged_RCC_s = overview_RCC[overview_RCC["source"] == name] 
            subsetmerged_RCC_t = overview_RCC[overview_RCC["target"] == name]
        
            subsetmerged_MF_s = overview_MF[overview_MF["source"] == name]
            subsetmerged_MF_t = overview_MF[overview_MF["target"] == name]
             


        if intra_inter == "intra":
            print("RCC")
            pycrosstalker.pl.plot_sankey(subsetmerged_RCC_s,threshold = 100, plt_name =None)
            print("MF")
            pycrosstalker.pl.plot_sankey(subsetmerged_MF_s, threshold=100, plt_name = None)

        elif intra_inter == "inter":
            if experimental == "yes":
       
                #df_cluster = merged_lr[merged_lr['cluster_t'] == name]
                print("RCC")
                plot_sankey(subsetmerged_RCC, target_cluster = name, threshold =50, plt_name = None)
                #plot_sankey(subsetmerged_RCC_t,threshold = 100, plt_name = f"RCC  {name}")
                #print("MF")
                #plot_sankey(subsetmerged_MF_s, threshold=100, plt_name = f"MF {name}")
                #plot_sankey(subsetmerged_MF_t, threshold=100, plt_name = f"MF {name}")
            else:
                print("RCC")
                pycrosstalker.pl.plot_sankey(subsetmerged_RCC_s, threshold =50, plt_name =None)
                pycrosstalker.pl.plot_sankey(subsetmerged_RCC_t,threshold = 50, plt_name = None)
                print("MF")
                pycrosstalker.pl.plot_sankey(subsetmerged_MF_s, threshold=50, plt_name = None)
                pycrosstalker.pl.plot_sankey(subsetmerged_MF_t, threshold=50, plt_name =None)


    return subsetmerged, subsetmerged_RCC, subsetmerged_MF   


In [ ]:
def GO_stuff(opt, opt2, opt3, which_os = "linux", overview = None, experimental = None):
    f, clusters_cleaned = get_intra_inter(opt = opt, opt2= opt2, opt3= opt3)
    df_full, top_terms_list, top_terms_list_MF = plot_GO(f)
    subsetmerged, subsetmerged_RCC, subsetmerged_MF = plot_unique_bp_sankeys(f, clusters_cleaned, df_full, top_terms_list, intra_inter = opt3, which_os = which_os, overview = overview,  experimental = experimental)
    subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs = plot_unique_mf_sankeys(f, clusters_cleaned, df_full, top_terms_list_MF, intra_inter = opt3, which_os = which_os, overview = overview,  experimental = experimental)
 
    return df_full, top_terms_list, subsetmerged, subsetmerged_RCC, subsetmerged_MF, subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs

In [ ]:
df_full_intra, top_terms_list_intra, subsetmerged_intra, subsetmerged_RCC_intra, subsetmerged_MF_intra, subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs = GO_stuff(opt = "unique", opt2="a", opt3= "intra", which_os = which_os, overview  = "False")


In [ ]:
subsetmerged_RCC_intra

In [ ]:
df_full_inter, top_terms_list_inter, subsetmerged_inter, subsetmerged_RCC_inter, subsetmerged_MF_inter, subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs  = GO_stuff(opt="unique", opt2="a2", opt3= "inter", which_os = which_os, overview = "no", experimental = "fuck off")

In [ ]:
df_full_inter, top_terms_list_inter, subsetmerged_inter, subsetmerged_RCC_inter, subsetmerged_MF_inter, subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs  = GO_stuff(opt="unique", opt2="a2", opt3= "inter", which_os = "windows11", overview = "yes", experimental = "fuck off")

In [ ]:
#overview
df_full_inter, top_terms_list_inter, subsetmerged_inter, subsetmerged_RCC_inter, subsetmerged_MF_inter, subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs = GO_stuff(opt="unique", opt2="a2", opt3= "inter", which_os = which_os, overview = "yes", experimental = "fuck off")

In [ ]:
#experimental
df_full_inter, top_terms_list_inter, subsetmerged_inter, subsetmerged_RCC_inter, subsetmerged_MF_inter = GO_stuff(opt="unique", opt2="a2", opt3= "inter", which_os = which_os, overview = False, experimental = "yes")